In [3]:
%matplotlib inline

#Imports
import json
import matplotlib.pyplot as plt
import psycopg2
import pandas as pd
import numpy as np
import seaborn as sns

In [17]:
#Create connection with our database
#Steps
#Load config file
with open("../scripts/config.json",'r') as f:
    config = json.load(f)

#Create connection string
conn_string = 'host={} dbname={} user={} password={}'.format(
                    config.get("host"),
                    config.get("database"),
                    config.get("user"),
                    config.get("passw"))
#Create connection
conn = psycopg2.connect(conn_string)

In [18]:
#Read the organisation data
#We'll read it in chunks
chunk= 500

#Create sql query string
sql_query_str="""
    SELECT
        *
    FROM
        gtr.organisations      
"""

#Read the data
org_df = pd.read_sql(sql_query_str,
                 conn)

In [19]:
org_df.head()

,pkey,addresses,created,href,id,links,name
0,1,"{u'address': [{u'city': u'Newton Aycliffe', u'...",2016-03-31 18:16:26,http://gtr.rcuk.ac.uk:80/gtr/api/organisations...,BAB46AB6-90FC-4145-B885-0153BEA15A73,"{u'link': [{u'otherAttributes': {}, u'href': u...",A-Tech Fabrications Limited
1,2,{u'address': []},2016-03-31 18:16:26,http://gtr.rcuk.ac.uk:80/gtr/api/organisations...,BB979A0C-0F8D-4B3F-A8D0-06A5F0D45FB0,"{u'link': [{u'otherAttributes': {}, u'href': u...",Life Sciences
2,3,"{u'address': [{u'created': 1459448184000, u're...",2016-03-31 18:16:26,http://gtr.rcuk.ac.uk:80/gtr/api/organisations...,BCD9069A-6739-4039-86DB-042CB13254CB,"{u'link': [{u'otherAttributes': {}, u'href': u...",Zurich Insurance plc
3,4,"{u'address': [{u'created': 1459448184000, u'co...",2016-03-31 18:16:26,http://gtr.rcuk.ac.uk:80/gtr/api/organisations...,B69FACF5-5AB9-4D97-88CD-0A38C32935FF,"{u'link': [{u'otherAttributes': {}, u'href': u...",Limagrain UK Ltd
4,5,"{u'address': [{u'created': 1459448184000, u'co...",2016-03-31 18:16:26,http://gtr.rcuk.ac.uk:80/gtr/api/organisations...,B76FE143-1E07-40B0-8932-028207296A78,"{u'link': [{u'otherAttributes': {}, u'href': u...",China Earthquake Administration


In [20]:
# Need to filter and label organisations - Find universities
def find_uni(x):
    if "UNIV" in x.upper():
        return True
    elif "COLLEGE" in x.upper():
        return True
    else:
        return False
# apply function
org_df["uni_flag"] = org_df["name"].apply(lambda x: find_uni(x))

In [21]:
# Check counts
org_df.groupby("uni_flag")["uni_flag"].count()

uni_flag
False    23993
True      1572
Name: uni_flag, dtype: int64

In [22]:
df_orgs_check = org_df[org_df["uni_flag"] == True]
# uni_ids 
uni_ids = list(df_orgs_check["id"])
#check names
list(df_orgs_check["name"])

['Maastricht University',
 'Australian National University',
 "Cardiff and Vale University Health Board (Bwrdd Iechyd Prifysgol Caerdydd a'r Fro)",
 'Medical University of Graz',
 'University of Queensland',
 'Kyoto University',
 'Sao Paulo State University UNESP',
 'University of Chicago',
 'National Chung Hsing University',
 'Victoria University of Wellington',
 'Leeds Beckett University',
 'University of Dresden',
 'Auburn University',
 'Strathmore University',
 'University of  Caxias do Sul',
 'College of Medicine',
 'Ho Hai University',
 'University of North Dakota',
 'Peking University',
 'Medical University of Lodz',
 'The University of Strathclyde',
 'University of Birmingham',
 'Orkney College',
 'Claude Bernard University Lyon 1 (UCBL)',
 'University of Eastern Piedmont Amedeo Avogadro',
 'Ahmadu Bello University',
 'Autonomous University of Madrid (Universidad Aut\xc3\xb3noma de Madrid UAM)',
 'University of Wales, Newport',
 'Technical University of Munich',
 'Mississippi S

In [23]:
len(uni_ids)

1572

In [24]:
# get tidy project_org data
# read in json file
with open("../notebooks/project_org.json",'r') as f:
    project_org = json.load(f)

In [16]:
#check format
project_org[0]

{u'orgs': [{u'lad': u'E07000121',
   u'org_id': u'B151FD07-45A4-41E6-8CF8-441EE5F96CFF'}],
 u'project_id': u'00006CB7-61E0-4946-B7DB-DAE09ED63DE4',
 u'topics': nan}

In [13]:
# import LAD to countries lookup
with open("../notebooks/LAD15_CTRY15_UK_LU.csv",'r') as f:
    df_lad_county = pd.read_csv(f)
#df_lad_county.head()
# set to dict for lookup
d_lad_country = df_lad_county[["LAD15CD","CTRY15NM"]].set_index("LAD15CD")["CTRY15NM"].to_dict()
#d_lad_country

{'95A': 'Northern Ireland',
 '95B': 'Northern Ireland',
 '95C': 'Northern Ireland',
 '95D': 'Northern Ireland',
 '95E': 'Northern Ireland',
 '95F': 'Northern Ireland',
 '95G': 'Northern Ireland',
 '95H': 'Northern Ireland',
 '95I': 'Northern Ireland',
 '95J': 'Northern Ireland',
 '95K': 'Northern Ireland',
 '95L': 'Northern Ireland',
 '95M': 'Northern Ireland',
 '95N': 'Northern Ireland',
 '95O': 'Northern Ireland',
 '95P': 'Northern Ireland',
 '95Q': 'Northern Ireland',
 '95R': 'Northern Ireland',
 '95S': 'Northern Ireland',
 '95T': 'Northern Ireland',
 '95U': 'Northern Ireland',
 '95V': 'Northern Ireland',
 '95W': 'Northern Ireland',
 '95X': 'Northern Ireland',
 '95Y': 'Northern Ireland',
 '95Z': 'Northern Ireland',
 'E06000001': 'England',
 'E06000002': 'England',
 'E06000003': 'England',
 'E06000004': 'England',
 'E06000005': 'England',
 'E06000006': 'England',
 'E06000007': 'England',
 'E06000008': 'England',
 'E06000009': 'England',
 'E06000010': 'England',
 'E06000011': 'England

In [28]:
# label project if collaboration between multiple orgs
proj_collab = {}
# label project if collaboration with university
proj_uni = {}
# label project if collab with another country
proj_country = {}
for proj in project_org:
    org_count = 0
    uni_count = 0
    int_count = 0
    country = ""
    pcountry= ""
    for org in proj["orgs"]:
        org_count = org_count+1
        country = d_lad_country[org["lad"]]
        if pcountry!= "" and country!=pcountry:
            int_count = int_count + 1
        pcountry = country
        try:
            uni_ids.index(org["org_id"])
            uni_count = uni_count+1
        except:
            uni_count = uni_count
    if org_count>1:
        collab = 1
    else:
        collab = 0
    if uni_count>0:
        uni = 1
    else:
        uni = 0
    if int_count>0:
        international = 1
    else:
        international = 0
    proj_collab[proj["project_id"]] = collab
    proj_uni[proj["project_id"]] = uni
    proj_country[proj["project_id"]] = international

In [30]:
#check
#proj_country["003E6B05-0738-4BCF-B8C5-C56DE5F9CAF0"]
sum(proj_country.values())

2771

In [31]:
# create a number of dictionaries to:
#       store LAD
#       count number projects organisation has done
#       count number of projects organisation collaborated on with another org
#       count number of projects organisation collaborated on with another org outside of country
orgs_lads_dict = {}
orgs_pcount_dict = {}
orgs_ccount_dict = {}
orgs_ucount_dict = {}
orgs_icount_dict = {}
for proj in project_org:
    collab = proj_collab[proj["project_id"]]
    uni = proj_uni[proj["project_id"]]
    international = proj_country[proj["project_id"]]
    for org in proj["orgs"]:
        orgs_lads_dict[org["org_id"]]= org["lad"]
        #count projects per org
        try:
            orgs_pcount_dict[org["org_id"]] = orgs_pcount_dict[org["org_id"]] + 1           
        except KeyError:
            orgs_pcount_dict[org["org_id"]] = 1   
        #count collabs per org
        try:
            orgs_ccount_dict[org["org_id"]] = orgs_ccount_dict[org["org_id"]] + collab
        except KeyError:
            orgs_ccount_dict[org["org_id"]] = collab 
        #count if proj with university
        try:
            orgs_ucount_dict[org["org_id"]] = orgs_ucount_dict[org["org_id"]] + uni
        except KeyError:
            orgs_ucount_dict[org["org_id"]] = uni
        #count if proj with international collab
        try:
            orgs_icount_dict[org["org_id"]] = orgs_icount_dict[org["org_id"]] + international
        except KeyError:
            orgs_icount_dict[org["org_id"]] = international    

In [32]:
#check
#orgs_lads_dict["3A5E126D-C175-4730-9B7B-E6D8CF447F83"]
#orgs_pcount_dict["3A5E126D-C175-4730-9B7B-E6D8CF447F83"]
#orgs_ccount_dict["3A5E126D-C175-4730-9B7B-E6D8CF447F83"]
#orgs_ucount_dict["3A5E126D-C175-4730-9B7B-E6D8CF447F83"]
orgs_icount_dict["3A5E126D-C175-4730-9B7B-E6D8CF447F83"]

114

In [35]:
# convert dicts to series
slads = pd.Series(orgs_lads_dict, name="LADs")
spcount = pd.Series(orgs_pcount_dict, name="Proj_count")
sccount = pd.Series(orgs_ccount_dict, name="collab_count")
ucount = pd.Series(orgs_ucount_dict, name="uni_count")
icount = pd.Series(orgs_icount_dict, name="international_count")
# check
icount.head()

0003EB78-07B9-400E-AE39-2AD612C28612    0
000403BF-769B-412D-BCF4-5963D425DF40    0
000557E6-19A4-44E6-BBCC-D32E1DA9530D    0
0005728F-26ED-465D-8DCE-C49F181EDB2E    0
0015E7C8-5920-4494-B1E9-BA6502ECA2B6    1
Name: international_count, dtype: int64

In [36]:
#combine into df
df_orgs1 = pd.concat([slads,spcount,sccount,ucount,icount], axis=1).reset_index()
df_orgs1.head()

,index,LADs,Proj_count,collab_count,uni_count,international_count
0,0003EB78-07B9-400E-AE39-2AD612C28612,E06000056,1,1,1,0
1,000403BF-769B-412D-BCF4-5963D425DF40,E09000001,1,1,1,0
2,000557E6-19A4-44E6-BBCC-D32E1DA9530D,E06000018,3,1,0,0
3,0005728F-26ED-465D-8DCE-C49F181EDB2E,E07000165,1,1,1,0
4,0015E7C8-5920-4494-B1E9-BA6502ECA2B6,S12000035,1,1,1,1


In [37]:
# add org name
df_orgs2 = pd.merge(df_orgs1, org_df[["id","name","uni_flag"]], how="left", left_on="index", right_on="id")
df_orgs2 = df_orgs2.drop("id",axis=1)

In [38]:
df_orgs2.head()

,index,LADs,Proj_count,collab_count,uni_count,international_count,name,uni_flag
0,0003EB78-07B9-400E-AE39-2AD612C28612,E06000056,1,1,1,0,Wentworth Laboratories Ltd,False
1,000403BF-769B-412D-BCF4-5963D425DF40,E09000001,1,1,1,0,Deloitte MCS Limited,False
2,000557E6-19A4-44E6-BBCC-D32E1DA9530D,E06000018,3,1,0,0,Voice Key Limited,False
3,0005728F-26ED-465D-8DCE-C49F181EDB2E,E07000165,1,1,1,0,Duchy Parade Films,False
4,0015E7C8-5920-4494-B1E9-BA6502ECA2B6,S12000035,1,1,1,1,Fusion Marine Limited,False


In [39]:
# check project count distribution; 75% of organisations have only done 1 or 2 projects
df_orgs2["Proj_count"].describe()

count    16759.000000
mean         5.758816
std         64.755009
min          1.000000
25%          1.000000
50%          1.000000
75%          2.000000
max       2906.000000
Name: Proj_count, dtype: float64

In [100]:
# check collaboration count distribution; 75% of organisations have only collborate on 1 or 2 projects, 25% have never collaborated
df_orgs2["collab_count"].describe()

count    16759.000000
mean         2.864491
std         19.604621
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max        771.000000
Name: collab_count, dtype: float64

In [40]:
# filter to orgs that are unis/ have worked with unis
df_orgsu = df_orgs2[df_orgs2["uni_count"]>0]
# Check counts
df_orgsu.groupby("uni_flag")["uni_flag"].count()

uni_flag
False    9660
True      313
Name: uni_flag, dtype: int64

In [41]:
# still alot of organisations with low project counts
df_orgsu["Proj_count"].describe()

count    9973.000000
mean        8.785220
std        83.801257
min         1.000000
25%         1.000000
50%         1.000000
75%         3.000000
max      2906.000000
Name: Proj_count, dtype: float64

In [42]:
#export for play
df_orgsu.to_csv('orgs_collabs.csv', index=False)